![DataStax Academy](https://s3.amazonaws.com/datastaxtraining/vq8Jr36Gk48v/datastax-academy.svg "DataStax Academy")

# Exercise 8 - Cassandra Connector: Cassandra Convert Data

## Background

This exercise will look at the different methods for retrieving and converting rows from Cassandra into different objects, such as tuples and case classes.

You'll be making use of the `videos_by_year_title` table, which is:

There are certain columns, such as `avg_rating`, where the value can be null.

***

## Directions

#### 1. Create a case class for the `videos_by_year_title` table and use that to retrieve all of the movies added in 2009 whose description include the word 'dog'. Print out the title and description for each movie.

In [1]:
case class Video( added_year : Int, title : String, description : String ) 

val rows = sc.cassandraTable[Video]("killr_video", "videos_by_year_title").where("added_year=2009")
val filter = rows.filter(movie => movie.description.toLowerCase.contains("dog"))
val output = filter.map( movie => "Title: " + String.format("%1$-26s", movie.title) + "\n" +
                                  "Description: " + movie.description + "\n")
output.collect.foreach(println)

Title: A Dog Year                
Description: Jon Katz is close to burnout. He's a writer with writer's block; his wife has left for her sister's because he's emotionally distant; he rarely answers his phone. A kennel sends him a border collie that's undisciplined because of abuse. Despite a series of mishaps, Jon decides to keep trying with the dog, and he rents a dilapidated farm house to give the dog room to run. A local handyman refers Jon to a woman who might be able to help him train the dog. Reluctantly, Jon gives her a try. Is the dog the problem, or the owner?

Title: Hachi: A Dog's Tale       
Description: A drama based on the true story of a college professor's bond with the abandoned dog he takes into his home.

Title: Hotel for Dogs            
Description: Placed in a foster home that doesn't allow pets, 16-year-old Andi and her younger brother, Bruce, turn an abandoned hotel into a home for their dog. Soon other strays arrive, and the hotel becomes a haven for every orp

#### 2. Using one of the methods to convert Cassandra rows to tuples, determine the percentage of movies in 2014 that do not have a user rating.

In [3]:
val qry1 = sc.cassandraTable[(String,Option[Float])]("killr_video", "videos_by_year_title").where("added_year=2014")
val qry2 = qry1.select("title", "avg_rating")
val movieCount = qry2.count
val ratedCount = qry2.filter(movie => movie._2.isEmpty).count

printf("Percentage: %.2f%%", 100.0 * ratedCount / movieCount)

309 2
Percentage: 0.65%

Notice we keep the `where()` clause instead of moving the condition into `filter()` so that Cassandra natively queries the partition directly instead of pulling all the data into Scala and then filtering.